In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import MultinomialNB, BernoulliNB, ComplementNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
# Load the data 
file_path = "D:\Sourav_Singh\Excell ,SQL & Tableau\Project_to_make\Text_Preprocessing_Naive_Bays.csv"
data = pd.read_csv(file_path)

In [3]:
# dataset information 

def display_data_info(data):
    
    # Basic Information
    print("Dataset Information:\n" + "-" * 20 + "\n")
    
    # Shape of the dataset
    print(f"- Shape: {data.shape}\n")
    
    # Column Names
    print(f"- Columns: {data.columns.tolist()}\n")
    
    # First few rows
    print("- First Few Rows:\n", data.head(), "\n")
    
    # Statistical description
    print("- Description:\n", data.describe(include='all'), "\n") # Include all columns for a broad overview

In [4]:
# display the function
display_data_info(data)

Dataset Information:
--------------------

- Shape: (15499, 3)

- Columns: ['id', 'text', 'label']

- First Few Rows:
    id                                               text label
0   0  We started giving this to my 5 year old Labrad...  dogs
1   1  This product is easy to set up and use. I have...  dogs
2   2  my dog has nerves and wants to itch and chew.....  dogs
3   3  the breeded gave us a can of Nupro when we too...  dogs
4   4  Very study, well made poop bag. Easy to open a...  dogs 

- Description:
                   id     text  label
count   15499.000000    15499  15499
unique           NaN    15223      2
top              NaN  Length:   dogs
freq             NaN       32   9333
mean     8662.526808      NaN    NaN
std      5003.585739      NaN    NaN
min         0.000000      NaN    NaN
25%      4333.500000      NaN    NaN
50%      8658.000000      NaN    NaN
75%     12982.500000      NaN    NaN
max     17352.000000      NaN    NaN 



In [5]:
# View data
data

id                                               text label
0          0  We started giving this to my 5 year old Labrad...  dogs
1          1  This product is easy to set up and use. I have...  dogs
2          2  my dog has nerves and wants to itch and chew.....  dogs
3          3  the breeded gave us a can of Nupro when we too...  dogs
4          4  Very study, well made poop bag. Easy to open a...  dogs
...      ...                                                ...   ...
15494  17348  My dog loves these!! They soft and easy to bre...  dogs
15495  17349  My 7 month old rescue pup had parvo before I a...  dogs
15496  17350  Overall I had no problem cutting my cat's nail...  cats
15497  17351  We searched everywhere for something to keep o...  dogs
15498  17352  The biggest problem with this item is unreliab...  cats

[15499 rows x 3 columns]

In [6]:
# lowercasing text
data['text_cleaned'] = data['text'].str.lower()

In [7]:
# Encode labels into numerical format
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label'])

In [8]:
# Split dataset into training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(
    data['text_cleaned'], data['label_encoded'], test_size=0.2, random_state=42 )

In [9]:
# Applying Bag-of-Words and TF-IDF vectorization to transform text data

# 1. Bag-of-Words (BoW)
bow_vectorizer = CountVectorizer()
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_test_bow = bow_vectorizer.transform(X_test)

# 2. TF-IDF
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [10]:
# Summary of preprocessing output
preprocessing_info = { "X_train_shape_BoW": X_train_bow.shape,
                       "X_test_shape_BoW": X_test_bow.shape,
                       "X_train_shape_TFIDF": X_train_tfidf.shape,
                       "X_test_shape_TFIDF": X_test_tfidf.shape,
                       "label_classes": label_encoder.classes_     }

In [11]:
preprocessing_info

{'X_train_shape_BoW': (12399, 21483),
 'X_test_shape_BoW': (3100, 21483),
 'X_train_shape_TFIDF': (12399, 21483),
 'X_test_shape_TFIDF': (3100, 21483),
 'label_classes': array(['cats', 'dogs'], dtype=object)}

In [12]:
# Initialize vectorizer (I will Choose CountVectorizer over TfidfVectorizer )
vectorizer = CountVectorizer() 

# Fit training data and transform train and test sets
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [13]:
# Define the evaluation function

def evaluate_model_metrics(model, X_train, X_test, Y_train, Y_test):
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    
    # Calculate 
    accuracy = accuracy_score(Y_test, Y_pred)
    precision = precision_score(Y_test, Y_pred, average='weighted')
    recall = recall_score(Y_test, Y_pred, average='weighted')
    f1 = f1_score(Y_test, Y_pred, average='weighted')

    # Print metrics with the model name
    print(f"Model Evaluation Metrics : {model_name}")
    print("-" * 25)
    print(f"Accuracy : {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall   : {recall:.4f}")
    print(f"F1 Score : {f1:.4f}")
    print("\n")

    # Return the metrics as a dictionary
    return { "accuracy": accuracy,
             "precision": precision,
             "recall": recall,
             "f1_score": f1           }

In [14]:
# Initialize Naive Bayes models
models = {  "MultinomialNB_BoW": MultinomialNB(),
            "BernoulliNB_BoW": BernoulliNB(),
            "ComplementNB_BoW": ComplementNB(),
            "MultinomialNB_TFIDF": MultinomialNB(),
            "BernoulliNB_TFIDF": BernoulliNB(),
            "ComplementNB_TFIDF": ComplementNB()     }

In [15]:
# Evaluate each model using BoW and TF-IDF features
results = {}
for model_name, model in models.items():
    if 'BoW' in model_name:
        results[model_name] = evaluate_model_metrics(model, X_train_bow, X_test_bow, Y_train, Y_test)
    else:
        results[model_name] = evaluate_model_metrics(model, X_train_tfidf, X_test_tfidf, Y_train, Y_test)

Model Evaluation Metrics : MultinomialNB_BoW
-------------------------
Accuracy : 0.8458
Precision: 0.8452
Recall   : 0.8458
F1 Score : 0.8449


Model Evaluation Metrics : BernoulliNB_BoW
-------------------------
Accuracy : 0.7794
Precision: 0.7845
Recall   : 0.7794
F1 Score : 0.7714


Model Evaluation Metrics : ComplementNB_BoW
-------------------------
Accuracy : 0.8419
Precision: 0.8414
Recall   : 0.8419
F1 Score : 0.8416


Model Evaluation Metrics : MultinomialNB_TFIDF
-------------------------
Accuracy : 0.7735
Precision: 0.8109
Recall   : 0.7735
F1 Score : 0.7536


Model Evaluation Metrics : BernoulliNB_TFIDF
-------------------------
Accuracy : 0.7794
Precision: 0.7845
Recall   : 0.7794
F1 Score : 0.7714


Model Evaluation Metrics : ComplementNB_TFIDF
-------------------------
Accuracy : 0.8181
Precision: 0.8245
Recall   : 0.8181
F1 Score : 0.8125




In [16]:
# Set up a parameter grid for MultinomialNB's alpha hyperparameter
param_grid = {'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]}

In [17]:
# Initialize the MultinomialNB model and GridSearchCV
multinomial_nb = MultinomialNB()
grid_search = GridSearchCV(multinomial_nb, param_grid, scoring='accuracy', cv=5, n_jobs=-1)

In [18]:
# Perform grid search on Bag-of-Words features
grid_search.fit(X_train_bow, Y_train)

GridSearchCV(cv=5, estimator=MultinomialNB(), n_jobs=-1,
             param_grid={'alpha': [0.1, 0.5, 1.0, 1.5, 2.0]},
             scoring='accuracy')

In [19]:
# Extract the best model and its performance metrics
best_model = grid_search.best_estimator_
best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [20]:
# Evaluate the best model on the test set
Y_pred_best = best_model.predict(X_test_bow)
best_model_performance = {  "Best_Params": best_params,
                            "Test_Accuracy": float(accuracy_score(Y_test, Y_pred_best)),
                            "Test_Precision": float(precision_score(Y_test, Y_pred_best, average='weighted')),
                            "Test_Recall": float(recall_score(Y_test, Y_pred_best, average='weighted')),
                            "Test_f1_score": float(f1_score(Y_test, Y_pred_best, average='weighted'))            }

In [21]:
best_model_performance

{'Best_Params': {'alpha': 1.0},
 'Test_Accuracy': 0.8458064516129032,
 'Test_Precision': 0.8451739398511384,
 'Test_Recall': 0.8458064516129032,
 'Test_f1_score': 0.8449326071371597}